# CCS potentials with LAMMPS software

This example provides an illustration for usage of CCS spline potential  with LAMMPS. The spline potential is used via the pair_style table command in LAMMPS. The details on the  pair_style table command can be found on https://docs.lammps.org/pair_table.html. We will use the LAMMPS interface provided by ASE(https://wiki.fysik.dtu.dk/ase/ase/calculators/lammps.html)  to compare with the original CCS calculator. 

**Note**:
* The exponential head part of CCS potential is neglected.
* LAMMPS doesn't support one-body energy contibution (so onebody contributions  are set to 0). 
* LAMMPS spline table should have a smaller grid size to that used for CCS.

Firstly, we have to generate the LAMMPS readable spline table from CCS parameters. This is done as follows:

In [1]:
# Generates CCS.table file readable by LAMMPS
from ccs_fit.scripts.jsonTotable import asecalcTotable
tags=asecalcTotable("CCS_params.json",scale=10)  # scale controls the resolution of the gridsize; gridsize=dr/scale. 

The above command generates **CCS.table** file in LAMMPS readable format.

In [2]:
!head CCS.table


 O-O
 N 141 R 0.9799999999999999 2.38 

 1 0.9799999999999999 -2.2748396233964714 49.30165626523697
 2 0.9899999999999999 -2.7489776599871574 45.555863640001974
 3 0.9999999999999999 -3.1865551479385488 41.98954653737815
 4 1.0099999999999998 -3.589366842476754 38.60270495736555
 5 1.02 -3.959207498827901 35.39533889996402
 6 1.0299999999999998 -4.2978718722180735 32.36744836517381


The details on the header (second line of CCS.table) is provided by the tags variable.

In [3]:
tags

{'O-O': {'Rmin': 0.9799999999999999, 'Rcut': 2.38, 'dr': 0.01, 'N': 141}}

In [4]:
# import necessary modules
import json
import copy

from ase import Atoms
from ase.calculators.lammpsrun import LAMMPS
from ccs_fit.ase_calculator.ccs_ase_calculator import CCS
from ase.io import read,write
from ase.visualize import view
from ase.io.trajectory import Trajectory
import ase.units as units
from ase.optimize import BFGS

# Define parameters for LAMMPS and CCS calculator

In [5]:
with open ('CCS_params.json','r') as f:
    CCS_params=json.load(f)

CCS_params['One_body']['O']=0.0                           # Explicitly set the reference energies to 0.0

# Parameters for LAMMPS 
parameters = {
              'pair_style': 'table spline 1000',          # 1000 represents the number of points chosen for pre-interpolation by LAMMPS
              'pair_coeff': ['1 1  CCS.table O-O 2.38 '], # 2.38 is the Rcut value
              'command': '/usr/bin/lmp'                   # Remove or change to your local lammps executable
}


# Optimization of O<sub>2</sub> molecule

In [6]:
# Read data
struct=read('O2.POSCAR')

# Set up calculators
lammps = LAMMPS(parameters=parameters,keep_tmp_files=False)
ccs=CCS(CCS_params)

# Copy the structure for ccs calculation
struct_ccs=copy.deepcopy(struct)

# LAMMPS optimization
struct.calc = lammps
print("Energy from LAMMPS  pre optmization: ", struct.get_potential_energy())
traj = Trajectory('LAMMPS.traj', 'w', struct)
dyn=BFGS(struct)
dyn.attach(traj)
dyn.run(fmax=0.05)
print("Energy from LAMMPS after optimization: ", struct.get_potential_energy())

# CCS optimization
struct_ccs.calc=ccs
print("Energy from CCS calculator pre optmization:", struct_ccs.get_potential_energy())
traj = Trajectory('CCS.traj', 'w', struct_ccs)
dyn=BFGS(struct_ccs)
dyn.attach(traj)
dyn.run(fmax=0.05)
print("Energy from CCS calculator after optimization:", struct_ccs.get_potential_energy())


Energy from LAMMPS  pre optmization:  -1.691692137414474
      Step     Time          Energy         fmax
BFGS:    0 11:31:32       -1.691692        5.9855
BFGS:    1 11:31:32       -2.809430        7.0864
BFGS:    2 11:31:32       -5.951668        7.2622
BFGS:    3 11:31:32       -4.264915       32.6662
BFGS:    4 11:31:32       -6.431440        5.7687
BFGS:    5 11:31:32       -6.675708        4.0740
BFGS:    6 11:31:32       -6.765814        3.7175
BFGS:    7 11:31:32       -6.837344        0.8919
BFGS:    8 11:31:32       -6.842994        0.1341
BFGS:    9 11:31:32       -6.843117        0.0067
Energy from LAMMPS after optimization:  -6.843116848586797
Energy from CCS calculator pre optmization: -1.691692137414658
      Step     Time          Energy         fmax
BFGS:    0 11:31:32       -1.691692        5.9855
BFGS:    1 11:31:32       -2.809430        7.0864
BFGS:    2 11:31:32       -5.951668        7.2622
BFGS:    3 11:31:32       -4.264915       32.6662
BFGS:    4 11:31:32   

In [7]:
print ("Mean square error in energy after optimization between CCS and LAMMPS calculator:{}".format((struct.get_potential_energy()-struct_ccs.get_potential_energy())**2))

Mean square error in energy after optimization between CCS and LAMMPS calculator:1.169250853967286e-18
